# Basics of the DVR calculations with Libra

## Table of Content <a name="TOC"></a>

1. [General setups](#setups)
2. [Mapping points on multidimensional grids ](#mapping)
3. [Functions of the Wfcgrid2 class](#wfcgrid2)
4. [Showcase: computing energies of the HO eigenstates](#ho_showcase)
5. [Dynamics: computed with SOFT method](#soft_dynamics)

### A. Learning objectives

- to map sequential numbers of the grid points to the multi-dimensional index and vice versa
- to define the Wfcgrid2 class objects for DVR calculations
- to initialize wavefunctions of the grids
- to compute various properties of the wavefunctions defined on the grid
- to set up and conduct the quantum dynamics of the DVR of wavefunctions

### B. Use cases

- [Compute energies of the DVR wavefunctions](#energy-use-case)
- [Numerically exact solution of the TD-SE](#tdse-solution)

### C. Functions

- `liblibra::libdyn::libwfcgrid`  
  - [`compute_mapping`](#compute_mapping-1)
  - [`compute_imapping`](#compute_imapping-1)

### D. Classes and class members

- `liblibra::libdyn::libwfcgrid2`
  - [`Wfcgrid2`](#Wfcgrid2-1) | [also here](#Wfcgrid2-2)  
    - [`nstates`](#nstates-1)
    - [`ndof`](#ndof-1)  
    
    - [`Npts`](#Npts-1)  
    - [`npts`](#npts-1)  
    - [`rmin`](#rmin-1)  
    - [`rmax`](#rmax-1)  
    - [`dr`](#dr-1)  
    - [`kmin`](#kmin-1)  
    - [`dk`](#dk-1)  
    
    - [`gmap`](#gmap-1) | [also here](#gmap-2)
    - [`imap`](#imap-1) | [also here](#imap-2)  
    
    - [`PSI_dia`](#PSI_dia-1)  
    - [`reciPSI_dia`](#reciPSI_dia-1)      
    - [`PSI_adi`](#PSI_adi-1)  
    - [`reciPSI_adi`](#reciPSI_adi-1)  
    
    - [`Hdia`](#Hdia-1)  
    - [`U`](#U-1)      
        
    - [`add_wfc_Gau`](#add_wfc_Gau-1)
    - [`add_wfc_HO`](#add_wfc_HO-1) | [also here](#add_wfc_HO-2)
    - [`add_wfc_ARB`](#add_wfc_ARB-1)
    
    - [`norm`](#norm-1) | [also here](#norm-2)
    - [`e_kin`](#e_kin-1) | [also here](#e_kin-2)
    - [`e_pot`](#e_pot-1) | [also here](#e_pot-2)
    - [`e_tot`](#e_tot-1) | [also here](#e_tot-2)
    - [`get_pow_q`](#get_pow_q-1) 
    - [`get_pow_p`](#get_pow_p-1) | [also here](#e_kin-2)
    - [`get_den_mat`](#get_den_mat-1)
    - [`get_pops`](#get_pops-1) | [also here](#get_pops-2)
    
    - [`update_propagator_H`](#update_propagator_H-1) | [also here](#update_propagator_H-2)
    - [`update_propagator_K`](#update_propagator_K-1)
    - [`SOFT_propagate`](#SOFT_propagate-1)
    
    - [`update_reciprocal`](#update_reciprocal-1) | [also here](#update_reciprocal-2)    
    
    - [`normalize`](#normalize-1) | [also here](#normalize-2)
    
    - [`update_Hamiltonian`](#update_Hamiltonian-1) | [also here](#update_Hamiltonian-2)
    - [`update_adiabatic`](#update_adiabatic-1)
    

## 1. General setups 
<a name="setups"></a>[Back to TOC](#TOC)

First, import all the necessary libraries:
* liblibra_core - for general data types from Libra

The output of the cell below will throw a bunch of warnings, but this is not a problem nothing really serios. So just disregard them.

In [1]:
import os
import sys
import math
if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
from libra_py import data_outs


/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, false> > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<float, std::all

Also, lets import matplotlib for plotting and define all the plotting parameters: sizes, colors, etc.

In [2]:
import matplotlib.pyplot as plt   # plots

plt.rc('axes', titlesize=38)      # fontsize of the axes title
plt.rc('axes', labelsize=38)      # fontsize of the x and y labels
plt.rc('legend', fontsize=38)     # legend fontsize
plt.rc('xtick', labelsize=38)    # fontsize of the tick labels
plt.rc('ytick', labelsize=38)    # fontsize of the tick labels

plt.rc('figure.subplot', left=0.2)
plt.rc('figure.subplot', right=0.95)
plt.rc('figure.subplot', bottom=0.13)
plt.rc('figure.subplot', top=0.88)

colors = {}

colors.update({"11": "#8b1a0e"})  # red       
colors.update({"12": "#FF4500"})  # orangered 
colors.update({"13": "#B22222"})  # firebrick 
colors.update({"14": "#DC143C"})  # crimson   

colors.update({"21": "#5e9c36"})  # green
colors.update({"22": "#006400"})  # darkgreen  
colors.update({"23": "#228B22"})  # forestgreen
colors.update({"24": "#808000"})  # olive      

colors.update({"31": "#8A2BE2"})  # blueviolet
colors.update({"32": "#00008B"})  # darkblue  

colors.update({"41": "#2F4F4F"})  # darkslategray

clrs_index = ["11", "21", "31", "41", "12", "22", "32", "13","23", "14", "24"]

We'll use these auxiliary functions later:

In [3]:
class tmp:
    pass

def harmonic1D(q, params):
    """
    1D Harmonic potential 
    """
  
    x = q.get(0)
    k = params["k"]
   
    obj = tmp()
    obj.ham_dia = CMATRIX(1,1)    
    obj.ham_dia.set(0,0, 0.5*k*x**2)

    return obj
    

def harmonic2D(q, params):
    """
    2D Harmonic potential 
    """
  
    x = q.get(0)
    y = q.get(1)
    kx = params["kx"]
    ky = params["ky"]
   
    obj = tmp()
    obj.ham_dia = CMATRIX(1,1)    
    obj.ham_dia.set(0, 0, (0.5*kx*x**2 + 0.5*ky*y**2)*(1.0+0.0j) )

    return obj
    


## 2. Mapping points on multidimensional grids 
<a name="mapping"></a>[Back to TOC](#TOC)


Imagine a 3D grid with:
 * 3 points in the 1-st dimension
 * 2 points in the 2-nd dimension  
 * 4 points in the 3-rd dimension

So there are 3 x 2 x 4 = 24 points 
 
However, we can still store all of them in 1D array, which is more efficient way. However, to refer to the points, we need a function that does the mapping.

This example demonstrates the functions:

`vector<vector<int> > compute_mapping(vector<vector<int> >& inp, vector<int>& npts)`

`int compute_imapping(vector<int>& inp, vector<int>& npts)`

defined in:   dyn/wfcgrid/Grid_functions.h
<a name="compute_mapping-1"></a>

In [4]:
inp = intList2()
npts = Py2Cpp_int([3,2,4])

res = compute_mapping(inp, npts);

print("The number of points = ", len(res) )
print("The number of dimensions = ", len(res[0]) )


The number of points =  24
The number of dimensions =  3


And the inverse of that mapping
<a name="compute_imapping-1"></a>

In [5]:
cnt = 0
for i in res:
    print("point # ", cnt, Cpp2Py(i) )
    print("index of that point in the global array =", compute_imapping(i, Py2Cpp_int([3,2,4])) )
    cnt +=1

point #  0 [0, 0, 0]
index of that point in the global array = 0
point #  1 [0, 0, 1]
index of that point in the global array = 1
point #  2 [0, 0, 2]
index of that point in the global array = 2
point #  3 [0, 0, 3]
index of that point in the global array = 3
point #  4 [0, 1, 0]
index of that point in the global array = 4
point #  5 [0, 1, 1]
index of that point in the global array = 5
point #  6 [0, 1, 2]
index of that point in the global array = 6
point #  7 [0, 1, 3]
index of that point in the global array = 7
point #  8 [1, 0, 0]
index of that point in the global array = 8
point #  9 [1, 0, 1]
index of that point in the global array = 9
point #  10 [1, 0, 2]
index of that point in the global array = 10
point #  11 [1, 0, 3]
index of that point in the global array = 11
point #  12 [1, 1, 0]
index of that point in the global array = 12
point #  13 [1, 1, 1]
index of that point in the global array = 13
point #  14 [1, 1, 2]
index of that point in the global array = 14
point #  15 [1,

## 3. Functions of the Wfcgrid2 class
<a name="wfcgrid2"></a>[Back to TOC](#TOC)

This example demonstrates the functions of the class `Wfcgrid2`
       
 defined in:   `dyn/wfcgrid2/Wfcgrid2.h`


Here, we test simple Harmonic oscillator eigenfunctions and will 
compare the energies as computed by Libra to the analytic results

### 3.1. Initialize the grid and do the mappings (internally):

`Wfcgrid2(vector<double>& rmin_, vector<double>& rmax_, vector<double>& dr_, int nstates_)`
<a name="Wfcgrid2-1"></a>

In [6]:
num_el_st = 1
wfc = Wfcgrid2(Py2Cpp_double([-15.0]), Py2Cpp_double([15.0]),  Py2Cpp_double([0.01]), num_el_st)

The key descriptors are stored in the `wfc` object:
<a name="nstates-1"></a> <a name="ndof-1"></a> <a name="Npts-1"></a> <a name="npts-1"></a> 
<a name="rmin-1"></a> <a name="rmax-1"></a> <a name="dr-1"></a> <a name="kmin-1"></a> <a name="dk-1"></a>

In [7]:
print(F"number of quantum states: {wfc.nstates}")
print(F"number of nuclear degrees of freedom: {wfc.ndof}")
print(F"the total number of grid points: {wfc.Npts}")
print(F"the number of grid points in each dimension: {Cpp2Py(wfc.npts)}")
print(F"the lower boundary of the real-space grid in each dimension: {Cpp2Py(wfc.rmin)}")
print(F"the upper boundary of the real-space grid in each dimension: {Cpp2Py(wfc.rmax)}")
print(F"the real-space grid-step in each dimension: {Cpp2Py(wfc.dr)}")
print(F"the lower boundary of the reciprocal-space grid in each dimension: {Cpp2Py(wfc.kmin)}")
print(F"the reciprocal-space grid-step in each dimension: {Cpp2Py(wfc.dk)}")

number of quantum states: 1
number of nuclear degrees of freedom: 1
the total number of grid points: 4096
the number of grid points in each dimension: [4096]
the lower boundary of the real-space grid in each dimension: [-15.0]
the upper boundary of the real-space grid in each dimension: [15.0]
the real-space grid-step in each dimension: [0.01]
the lower boundary of the reciprocal-space grid in each dimension: [-50.0]
the reciprocal-space grid-step in each dimension: [0.0244140625]


### Exercise 1:

What is the upper boundary of reciprocal space?

Grid mapping : the wavefunctions are stored in a consecutive order.

To convert the single integer (which is just an order of the point in a real or reciprocal space) from 
the indices of the point on the 1D grid in each dimensions, we use the mapping below:
e.g. igmap[1] = [0, 1, 0, 0] means that the second (index 1) entry in the PSI array below corresponds to
a grid point that is first (lower boundary) in dimensions 0, 2, and 3, but is second (index 1) in the 
dimension 1. Same for the reciprocal space

<a name="gmap-1"></a>

In [8]:
for i in range(10):
    print(F"the point {i} corresponds to the grid indices = {Cpp2Py(wfc.gmap[i]) }")

the point 0 corresponds to the grid indices = [0]
the point 1 corresponds to the grid indices = [1]
the point 2 corresponds to the grid indices = [2]
the point 3 corresponds to the grid indices = [3]
the point 4 corresponds to the grid indices = [4]
the point 5 corresponds to the grid indices = [5]
the point 6 corresponds to the grid indices = [6]
the point 7 corresponds to the grid indices = [7]
the point 8 corresponds to the grid indices = [8]
the point 9 corresponds to the grid indices = [9]


Analogously, the inverse mapping of the indices of the point on the axes of all dimensions to the sequentian number:
<a name="imap-1"></a>

In [9]:
for i in range(10):
    print(F"the point {i} corresponds to the grid indices = { wfc.imap( Py2Cpp_int([i]) )   }")

the point 0 corresponds to the grid indices = 0
the point 1 corresponds to the grid indices = 1
the point 2 corresponds to the grid indices = 2
the point 3 corresponds to the grid indices = 3
the point 4 corresponds to the grid indices = 4
the point 5 corresponds to the grid indices = 5
the point 6 corresponds to the grid indices = 6
the point 7 corresponds to the grid indices = 7
the point 8 corresponds to the grid indices = 8
the point 9 corresponds to the grid indices = 9


### 3.2. Let's run the above examples for a 2D case:

<a name="Wfcgrid2-2"></a> <a name="gmap-2"></a> <a name="imap-2"></a>

In [10]:
wfc2 = Wfcgrid2(Py2Cpp_double([-15.0, -15.0]), Py2Cpp_double([15.0, 15.0]),  Py2Cpp_double([1, 1]), num_el_st)


print(F"number of quantum states: {wfc2.nstates}")
print(F"number of nuclear degrees of freedom: {wfc2.ndof}")
print(F"the total number of grid points: {wfc2.Npts}")
print(F"the number of grid points in each dimension: {Cpp2Py(wfc2.npts)}")
print(F"the lower boundary of the real-space grid in each dimension: {Cpp2Py(wfc2.rmin)}")
print(F"the upper boundary of the real-space grid in each dimension: {Cpp2Py(wfc2.rmax)}")
print(F"the real-space grid-step in each dimension: {Cpp2Py(wfc2.dr)}")
print(F"the lower boundary of the reciprocal-space grid in each dimension: {Cpp2Py(wfc2.kmin)}")
print(F"the reciprocal-space grid-step in each dimension: {Cpp2Py(wfc2.dk)}")

for i in range(10):
    print(F"the point {i} corresponds to the grid indices = {Cpp2Py(wfc2.gmap[i]) }")
    
for i in range(10):
    print(F"the point {i} corresponds to the grid indices = { wfc2.imap( Py2Cpp_int([i, i]) )   }")    

number of quantum states: 1
number of nuclear degrees of freedom: 2
the total number of grid points: 1024
the number of grid points in each dimension: [32, 32]
the lower boundary of the real-space grid in each dimension: [-15.0, -15.0]
the upper boundary of the real-space grid in each dimension: [15.0, 15.0]
the real-space grid-step in each dimension: [1.0, 1.0]
the lower boundary of the reciprocal-space grid in each dimension: [-0.5, -0.5]
the reciprocal-space grid-step in each dimension: [0.03125, 0.03125]
the point 0 corresponds to the grid indices = [0, 0]
the point 1 corresponds to the grid indices = [0, 1]
the point 2 corresponds to the grid indices = [0, 2]
the point 3 corresponds to the grid indices = [0, 3]
the point 4 corresponds to the grid indices = [0, 4]
the point 5 corresponds to the grid indices = [0, 5]
the point 6 corresponds to the grid indices = [0, 6]
the point 7 corresponds to the grid indices = [0, 7]
the point 8 corresponds to the grid indices = [0, 8]
the point

### 3.3. Add a wavefunction to the grid

This can be done by sequentially adding either Gaussian wavepackets or the Harmonic osccillator eigenfunctions to the grid with the corresponding weights.

Adding of such functions is done with for instance:

`void add_wfc_HO(vector<double>& x0, vector<double>& px0, vector<double>& alpha, int init_state, vector<int>& nu, complex<double> weight, int rep)`

Here,

* `x0` - is the center of the added function
* `p0` - it's initial momentum (if any)
* `alpha` - the exponent parameters 
* `init_state` - specialization of the initial electronic state
* `nu` - the selector of the HO eigenstate to be added
* `weight` - the amplitude with which the added function enters the superpositions, doesn't have to lead to a normalized function, the norm is included when computing the properties
* `rep` - representation

The variables x0, p0, etc. should have the dimensionality comparable to that of the grid. 

For instance, in the example below we add the wavefunction (single HO eigenstate) to the 1D grid
<a name="add_wfc_HO-1"></a> <a name="norm-1"></a>

In [11]:
x0 = Py2Cpp_double([0.0])            
p0 = Py2Cpp_double([0.0])            
alphas = Py2Cpp_double([1.0])
nu = Py2Cpp_int([0])
el_st = 0
rep = 0  

weight = 1.0+0.0j
wfc.add_wfc_HO(x0, p0, alphas, el_st, nu, weight, rep)

print(F" norm of the diabatic wfc = {wfc.norm(0)} and norm of the adiabatic wfc = {wfc.norm(1)}")

 norm of the diabatic wfc = 0.9999999999999986 and norm of the adiabatic wfc = 0.0


We can see that the wavefunction is pretty much normalized - this is becasue we have only added a single wavefunction which is already normalized.

Also, note how the norm of the diabatic wavefunction is 1.0, but that of the adiabatic is zero - this is because we have added the wavefunction only in the diabatic representation (`rep = 0`) and haven't yet run any calculations to do any updates of the other (adiabatic) representation

### Exercise 2
<a name="add_wfc_Gau-1"></a>
Use the `add_wfc_Gau` function to add several Gaussians to the grid.

### Exercise 3

Initialize the wavefunction as the superposition: $|0> - 0.5 |1> + 0.25i |2 >$

Is the resulting wavefunction normalized?

Use the `normalize()` method of the `Wfcgrid2` class to normalize it
<a name="normalize-1"></a>

### 3.4. A more advanced example: adding an arbitrary wavefunctions 

using the `add_wfc_ARB` method

All we need to do is to set up a Python function that would take `vector<double>` as the input for coordinates, a Python dictionary for parameters, and it would return a `CMATRIX(nstates, 1)` object containing energies of all states as the function of the multidimensional coordinate. 

Let's define the one:

In [12]:
def my_2D_sin(q, params):
    """
    2D sine potential 
    """
  
    x = q.get(0,0)
    y = q.get(1,0)
    A = params["A"]
    alpha = params["alpha"]
    omega = params["omega"]
   
    res = CMATRIX(1,1)    
    res.set(0,0, 0.5* A * math.sin(omega*(x**2 + y**2)) * math.exp(-alpha*(x**2 + y**2)) )

    return res


Now, we can add the wavefunction to that grid using:

`void add_wfc_ARB(bp::object py_funct, bp::object params, int rep)`
<a name="add_wfc_ARB-1"></a>

In [13]:
rep = 0  
wfc2.add_wfc_ARB(my_2D_sin, {"A":1, "alpha":1.0, "omega":1.0}, rep)

print(F" norm of the diabatic wfc = {wfc2.norm(0)} and norm of the adiabatic wfc = {wfc2.norm(1)}")

 norm of the diabatic wfc = 0.11124683022492163 and norm of the adiabatic wfc = 0.0


As we can see, this wavefunction is not normalized.

We can normalize it using `normalize(int rep)` method with `rep = 0` since we are working with the diabatic representation
<a name="normalize-2"></a>

In [14]:
wfc2.normalize(0)
print(F" norm of the diabatic wfc = {wfc2.norm(0)} and norm of the adiabatic wfc = {wfc2.norm(1)}")

 norm of the diabatic wfc = 0.9999999999999994 and norm of the adiabatic wfc = 0.0


### 3.5. Accessing wavefunction and the internal data

Now that we have initialized the wavefunction, we can access the wavefunction 
<a name="PSI_dia-1"></a> <a name="PSI_adi-1"></a>

In [15]:
for i in range(10):
    print(F"diabatic wfc = {wfc.PSI_dia[500+i].get(0,0) } adiabatic wfc = {wfc.PSI_adi[500+i].get(0,0) }")

diabatic wfc = (1.4487332796885729e-22+0j) adiabatic wfc = 0j
diabatic wfc = (1.6010178358670647e-22+0j) adiabatic wfc = 0j
diabatic wfc = (1.7691329616726924e-22+0j) adiabatic wfc = 0j
diabatic wfc = (1.954705561969595e-22+0j) adiabatic wfc = 0j
diabatic wfc = (2.159527773482247e-22+0j) adiabatic wfc = 0j
diabatic wfc = (2.3855735423596517e-22+0j) adiabatic wfc = 0j
diabatic wfc = (2.6350168440629746e-22+0j) adiabatic wfc = 0j
diabatic wfc = (2.91025170616495e-22+0j) adiabatic wfc = 0j
diabatic wfc = (3.2139142101365356e-22+0j) adiabatic wfc = 0j
diabatic wfc = (3.5489066651617274e-22+0j) adiabatic wfc = 0j


We can also see what the reciprocal of the wavefunctions are. 
<a name="reciPSI_dia-1"></a> <a name="reciPSI_adi-1"></a>

In [16]:
for i in range(10):
    print(F"diabatic wfc = {wfc.reciPSI_dia[500+i].get(0,0) } adiabatic wfc = {wfc.reciPSI_adi[500+i].get(0,0) }")

diabatic wfc = 0j adiabatic wfc = 0j
diabatic wfc = 0j adiabatic wfc = 0j
diabatic wfc = 0j adiabatic wfc = 0j
diabatic wfc = 0j adiabatic wfc = 0j
diabatic wfc = 0j adiabatic wfc = 0j
diabatic wfc = 0j adiabatic wfc = 0j
diabatic wfc = 0j adiabatic wfc = 0j
diabatic wfc = 0j adiabatic wfc = 0j
diabatic wfc = 0j adiabatic wfc = 0j
diabatic wfc = 0j adiabatic wfc = 0j


### 3.6. Update the reciprocal of the initial wavefunction

This is needed for computing some properties, and also as the initialization of the dynamics
<a name="update_reciprocal-1"></a>

In [17]:
wfc.update_reciprocal(rep)

Now, since we have computed the reciprocal of the wavefunction (by doing an FFT of the real-space wfc), we can access those numbers (still in the diabatic representation only)

In [18]:
for i in range(10):
    print(F"diabatic wfc = {wfc.reciPSI_dia[500+i].get(0,0) } adiabatic wfc = {wfc.reciPSI_adi[500+i].get(0,0) }")

diabatic wfc = (-7.938581813042437e-16-3.292442409596794e-16j) adiabatic wfc = 0j
diabatic wfc = (-1.0534997694888374e-15-4.3324393460622627e-16j) adiabatic wfc = 0j
diabatic wfc = (-1.344645905336897e-15-5.169367280259415e-16j) adiabatic wfc = 0j
diabatic wfc = (-1.5875806328520155e-15-6.733472460693432e-16j) adiabatic wfc = 0j
diabatic wfc = (-1.9543907628057097e-15-8.294333772218843e-16j) adiabatic wfc = 0j
diabatic wfc = (-2.4186289504751935e-15-9.33707190095722e-16j) adiabatic wfc = 0j
diabatic wfc = (-2.872689724086602e-15-1.1414549034631344e-15j) adiabatic wfc = 0j
diabatic wfc = (-3.1856187839746186e-15-1.3084149947233198e-15j) adiabatic wfc = 0j
diabatic wfc = (-3.578929217190932e-15-1.5015963563073512e-15j) adiabatic wfc = 0j
diabatic wfc = (-3.923212091435534e-15-1.4885132888172904e-15j) adiabatic wfc = 0j


### 3.4. Compute the Hamiltonian on the grid

The nice thing is - we can define any Hamiltonian function right in Python (this is done in [section 1]() ) and pass that function, together with the dictionary of the corresponding parameters to the `update_Hamiltonian` method.

Here, we define the force constant of the potential to be consistent with the alpha of the initial Gaussian wavepacket and the mass of the particle, as is done in any Quantum chemistry textbooks.
<a name="update_Hamiltonian-1"></a> 

In [19]:
masses = Py2Cpp_double([2000.0])
omega = alphas[0]/masses[0]
k = masses[0] * omega**2

wfc.update_Hamiltonian(harmonic1D, {"k": k}, rep)

After this step, the internal storage will also contain the Hamitonians computed at the grid points:
<a name="Hdia-1"></a> 

In [20]:
for i in range(10):
    print(F"diabatic Hamiltonian (potential only) = {wfc.Hdia[500+i].get(0,0) } ")

diabatic Hamiltonian (potential only) = (0.025+0j) 
diabatic Hamiltonian (potential only) = (0.024950025+0j) 
diabatic Hamiltonian (potential only) = (0.0249001+0j) 
diabatic Hamiltonian (potential only) = (0.024850224999999997+0j) 
diabatic Hamiltonian (potential only) = (0.024800400000000004+0j) 
diabatic Hamiltonian (potential only) = (0.024750624999999995+0j) 
diabatic Hamiltonian (potential only) = (0.024700899999999998+0j) 
diabatic Hamiltonian (potential only) = (0.024651225000000002+0j) 
diabatic Hamiltonian (potential only) = (0.0246016+0j) 
diabatic Hamiltonian (potential only) = (0.024552025+0j) 


### 3.5. Computing properties

Now, when the Hamiltonian is evaluated on the grid, we can compute various properties.

In this example, we use the wavefunction represented in the diabatic basis
<a name="norm-2"></a> <a name="e_kin-1"></a> <a name="e_pot-1"></a> <a name="e_tot-1"></a> <a name="get_pow_p-1"></a> 

In [21]:
rep = 0
print( "Norm = ", wfc.norm(rep) )
print( "Ekin = ", wfc.e_kin(masses, rep) )
print( "Expected kinetic energy = ", 0.5*alphas[0]/(2.0*masses[0]) )
print( "Epot = ", wfc.e_pot(rep) )
print( "Expected potential energy = ", (0.5*k/alphas[0])*(0.5 + nu[0]) )
print( "Etot = ", wfc.e_tot(masses, rep) )
print( "Expected total energy = ", omega*(0.5 + nu[0]) )

p2 = wfc.get_pow_p(0, 2);
print( "p2 = ", p2.get(0).real )
print( "p2/2*m = ", p2.get(0).real/(2.0 * masses[0]) )

Norm =  0.9999999999999986
Ekin =  0.00012499999999999567
Expected kinetic energy =  0.000125
Epot =  0.00012499999999999987
Expected potential energy =  0.000125
Etot =  0.00024999999999999556
Expected total energy =  0.00025
p2 =  0.49999999999998224
p2/2*m =  0.00012499999999999556


We can also compute the populations of all states and resolve it by the spatial region too:

<a name="get_pops-1"></a> <a name="get_pops-2"></a> 

In [22]:
p = wfc.get_pops(0).get(0,0)
print(F" population of diabatic state 0 of wfc in the whole region  {p}")

left, right = Py2Cpp_double([-15.0]), Py2Cpp_double([0.0])
p = wfc.get_pops(0, left, right).get(0,0)
print(F" population of diabatic state 0 of wfc in the half of the original region  {p}")


 population of diabatic state 0 of wfc in the whole region  0.9999999999999986
 population of diabatic state 0 of wfc in the half of the original region  0.5028209479177388


### 3.6. Converting between diabatic and adiabatic representations

The transformation matrix `wfc.U` is computed when we compute the real-space propagator `wfc.update_propagator_H` 

For the purposes of the adi-to-dia transformation, it doesn't matter what value for dt is used in that function.

<a name="update_propagator_H-1"></a> 

In [23]:
wfc.update_propagator_H(0.0)

Now, we can access the transformation matrix - one for each grid point. 

Note, in this tutorial we deal with the 1 electronic state, so all the transformation matrices are just the identity ones
<a name="U-1"></a> 

In [24]:
for i in range(10):
    print(F"dia-to-adi transformation matrix at point {500+i}\n")
    data_outs.print_matrix(wfc.U[500+i])

dia-to-adi transformation matrix at point 500

(1+0j)  
dia-to-adi transformation matrix at point 501

(1+0j)  
dia-to-adi transformation matrix at point 502

(1+0j)  
dia-to-adi transformation matrix at point 503

(1+0j)  
dia-to-adi transformation matrix at point 504

(1+0j)  
dia-to-adi transformation matrix at point 505

(1+0j)  
dia-to-adi transformation matrix at point 506

(1+0j)  
dia-to-adi transformation matrix at point 507

(1+0j)  
dia-to-adi transformation matrix at point 508

(1+0j)  
dia-to-adi transformation matrix at point 509

(1+0j)  


Now, we can update the real-space adiabatic wavefunction and then its reciprocal for the adiabatic representation (`rep = 1`):

<a name="update_adiabatic-1"></a> <a name="update_reciprocal-1"></a> 

In [25]:
wfc.update_adiabatic()
wfc.update_reciprocal(1)

And compute the properties but now in the adiabatic basis

In [26]:
for i in range(10):
    print(F"diabatic wfc = {wfc.PSI_dia[500+i].get(0,0) } adiabatic wfc = {wfc.PSI_adi[500+i].get(0,0) }")

diabatic wfc = (1.4487332796885729e-22+0j) adiabatic wfc = (1.4487332796885729e-22+0j)
diabatic wfc = (1.6010178358670647e-22+0j) adiabatic wfc = (1.6010178358670647e-22+0j)
diabatic wfc = (1.7691329616726924e-22+0j) adiabatic wfc = (1.7691329616726924e-22+0j)
diabatic wfc = (1.954705561969595e-22+0j) adiabatic wfc = (1.954705561969595e-22+0j)
diabatic wfc = (2.159527773482247e-22+0j) adiabatic wfc = (2.159527773482247e-22+0j)
diabatic wfc = (2.3855735423596517e-22+0j) adiabatic wfc = (2.3855735423596517e-22+0j)
diabatic wfc = (2.6350168440629746e-22+0j) adiabatic wfc = (2.6350168440629746e-22+0j)
diabatic wfc = (2.91025170616495e-22+0j) adiabatic wfc = (2.91025170616495e-22+0j)
diabatic wfc = (3.2139142101365356e-22+0j) adiabatic wfc = (3.2139142101365356e-22+0j)
diabatic wfc = (3.5489066651617274e-22+0j) adiabatic wfc = (3.5489066651617274e-22+0j)


In [27]:
for i in range(10):
    print(F"diabatic wfc = {wfc.reciPSI_dia[500+i].get(0,0) } adiabatic wfc = {wfc.reciPSI_adi[500+i].get(0,0) }")

diabatic wfc = (-7.938581813042437e-16-3.292442409596794e-16j) adiabatic wfc = (-7.938581813042437e-16-3.292442409596794e-16j)
diabatic wfc = (-1.0534997694888374e-15-4.3324393460622627e-16j) adiabatic wfc = (-1.0534997694888374e-15-4.3324393460622627e-16j)
diabatic wfc = (-1.344645905336897e-15-5.169367280259415e-16j) adiabatic wfc = (-1.344645905336897e-15-5.169367280259415e-16j)
diabatic wfc = (-1.5875806328520155e-15-6.733472460693432e-16j) adiabatic wfc = (-1.5875806328520155e-15-6.733472460693432e-16j)
diabatic wfc = (-1.9543907628057097e-15-8.294333772218843e-16j) adiabatic wfc = (-1.9543907628057097e-15-8.294333772218843e-16j)
diabatic wfc = (-2.4186289504751935e-15-9.33707190095722e-16j) adiabatic wfc = (-2.4186289504751935e-15-9.33707190095722e-16j)
diabatic wfc = (-2.872689724086602e-15-1.1414549034631344e-15j) adiabatic wfc = (-2.872689724086602e-15-1.1414549034631344e-15j)
diabatic wfc = (-3.1856187839746186e-15-1.3084149947233198e-15j) adiabatic wfc = (-3.1856187839746186

In [28]:
print( "Norm = ", wfc.norm(1) )
print( "Ekin = ", wfc.e_kin(masses, 1) )
print( "Expected kinetic energy = ", 0.5*alphas[0]/(2.0*masses[0]) )
print( "Epot = ", wfc.e_pot(1) )
print( "Expected potential energy = ", (0.5*k/alphas[0])*(0.5 + nu[0]) )
print( "Etot = ", wfc.e_tot(masses, 1) )
print( "Expected total energy = ", omega*(0.5 + nu[0]) )

p2 = wfc.get_pow_p(1, 2);
print( "p2 = ", p2.get(0).real )
print( "p2/2*m = ", p2.get(0).real/(2.0 * masses[0]) )

Norm =  0.9999999999999986
Ekin =  0.00012499999999999567
Expected kinetic energy =  0.000125
Epot =  0.00012499999999999987
Expected potential energy =  0.000125
Etot =  0.00024999999999999556
Expected total energy =  0.00025
p2 =  0.49999999999998224
p2/2*m =  0.00012499999999999556


In [29]:
p = wfc.get_pops(1).get(0,0)
print(F" population of adiabatic state 0 of wfc in the whole region  {p}")

left, right = Py2Cpp_double([-15.0]), Py2Cpp_double([0.0])
p = wfc.get_pops(1, left, right).get(0,0)
print(F" population of adiabatic state 0 of wfc in the half of the original region  {p}")


 population of adiabatic state 0 of wfc in the whole region  0.9999999999999986
 population of adiabatic state 0 of wfc in the half of the original region  0.5028209479177388


## 4. Showcase: computing energies of the HO eigenstates
<a name="ho_showcase"></a>[Back to TOC](#TOC)
<a name="energy-use-case"></a>

We, of course, know all the properties of the HO eigenstates analytically. Namely, the energies should be:

\\[ E_n = \hbar \omega (n + \frac{1}{2}) \\]

Let's see if we can also get them numerically

In [30]:
for n in [0, 1, 2, 3, 10, 20]:

    wfc = Wfcgrid2(Py2Cpp_double([-15.0]), Py2Cpp_double([15.0]),  Py2Cpp_double([0.01]), num_el_st)
    
    nu = Py2Cpp_int([n]) 
    wfc.add_wfc_HO(x0, p0, alphas, el_st, nu, 1.0+0.0j, rep)

    wfc.update_reciprocal(rep)
    wfc.update_Hamiltonian(harmonic1D, {"k": k}, rep)

    print( "========== State %i ==============" % (n) )
    print( "Etot = ", wfc.e_tot(masses, rep) )
    print( "Expected total energy = ", omega*(0.5 + nu[0]) )



========== State 0 ==============
Etot =  0.00024999999999999556
Expected total energy =  0.00025
========== State 1 ==============
Etot =  0.000749999999999987
Expected total energy =  0.00075
========== State 2 ==============
Etot =  0.00124999999999998
Expected total energy =  0.00125
========== State 3 ==============
Etot =  0.0017499999999999684
Expected total energy =  0.00175
========== State 10 ==============
Etot =  0.0052499999999999075
Expected total energy =  0.00525
========== State 20 ==============
Etot =  0.01024999999999985
Expected total energy =  0.01025


## 5. Dynamics: computed with SOFT method
<a name="soft_dynamics"></a>[Back to TOC](#TOC)
<a name="tdse-solution"></a>

### 5.1. Initialization

As usual, let's initialize the grid and populate it with some wavefunction

In this case, we start with a superposition of 2 HO eigenstates, so the initial wavefunction is not stationary with respect ot the chosen potential (or we won't be able to see any dynamics)

As in the axamples above, we update the reciprocal wavefunction and then the Hamiltonian

<a name="add_wfc_HO-2"></a> <a name="update_reciprocal-2"> <a name="update_Hamiltonian-2"></a>

In [31]:
wfc = Wfcgrid2(Py2Cpp_double([-15.0]), Py2Cpp_double([15.0]),  Py2Cpp_double([0.01]), num_el_st)
    
wfc.add_wfc_HO(x0, p0, alphas, el_st, Py2Cpp_int([0]) , 1.0+0.0j, rep)
wfc.add_wfc_HO(x0, p0, alphas, el_st, Py2Cpp_int([1]) , 1.0+0.0j, rep)

wfc.update_reciprocal(rep)
wfc.update_Hamiltonian(harmonic1D, {"k": k}, rep)

### 5.2. Update the propagators 

To compute the quantum dynamics on the grid, all we need to do is first to compute the propagators - the matrices that advances the wavefunction in real and reciprocal spaces.

The split-operator Fourier-transform (SOFT) method dates back to Kosloff & Kosloff and is basically the following:

If the Hamiltonian is given by:

\\[  H = K + V  \\]

Then, the solution of the TD-SE:

\\[  i \hbar \frac{\partial \psi}{\partial t} = H \psi \\]

is given by:

\\[ \psi(t) = exp(-i \frac{H t}{\hbar} ) \psi(0)  \\]

Of course, in practice we compute the state advancement by only small time increment \\[ \Delta t \\] as:

\\[ \psi(t + \Delta t) = exp(-i \frac{H \Delta t}{\hbar} ) \psi(t)  \\]

So it all boils down to the computing the propagator 

\\[ exp(-i \frac{H \Delta t}{\hbar} ) \\]

This is then done by the Trotter splitting technique:

\\[ exp(-i \frac{H \Delta t}{\hbar} ) \approx exp(-i \frac{V \Delta t}{2 \hbar} ) exp(-i \frac{K \Delta t}{\hbar} ) exp(-i \frac{V \Delta t}{2 \hbar} )  \\]


In the end, we need to compute the operators $ exp(-i \frac{V \Delta t}{2 \hbar} ) $ and 
$exp(-i \frac{K \Delta t}{\hbar} )$

This is done by:

<a name="update_propagator_H-2"></a> <a name="update_propagator_K-1"></a>

In [32]:
dt = 10.0  
wfc.update_propagator_H(0.5*dt)
wfc.update_propagator_K(dt, masses)

### 5.3. Compute the dynamics

The propagators in real and reciprocal spaces are stored in the class object, so we can now simply apply them many times to our starting wavefunction:

This is done with the `SOFT_propagate()` function.

Note how we use the following functions to compute the corresponding properties:

* `get_pow_q` - for \<q\>
* `get_pow_p` - for \<p\>
* `get_den_mat` - for $\rho_{ij} = |i><j|$
and so on


By default, the dynamics is executed in the diabatic representation, so for us to access the adiabatic properties (e.g. populations of the adiabatic states), we convert the propagated wavefunctions to the adiabatic representation with

* `update_adiabatic`

<a name="get_pow_q-1"></a> <a name="get_pow_p-2"></a> <a name="get_den_mat-1"></a> 
<a name="e_kin-2"></a> <a name="e_pot-2"></a> <a name="e_tot-2"></a> <a name="SOFT_propagate-1"> </a>

In [33]:
nsteps = 100
for step in range(nsteps):
    wfc.SOFT_propagate()
    q = wfc.get_pow_q(0, 1).get(0).real
    p = wfc.get_pow_p(0, 1).get(0).real
    
    # Diabatic is the rep used for propagation, so we need to 
    # convert wfcs into adiabatic one
    wfc.update_adiabatic()

    Ddia = wfc.get_den_mat(0)  # diabatic density matrix
    Dadi = wfc.get_den_mat(1)  # adiabatic density matrix

    p0_dia = Ddia.get(0,0).real
    p0_adi = Dadi.get(0,0).real
    print("step= ", step, " Ekin= ", wfc.e_kin(masses, rep), 
          " Epot= ", wfc.e_pot(rep), " Etot= ", wfc.e_tot(masses, rep), 
          " q= ", q, " p= ", p, " p0_dia= ", p0_dia, " p0_adi= ", p0_adi )

step=  0  Ekin=  0.00025000156250001165  Epot=  0.00025000000003912  Etot=  0.0005000015625391317  q=  0.7070979423518663  p=  -0.0017677669529488457  p0_dia=  1.0  p0_adi=  1.0
step=  1  Ekin=  0.00025000156242188827  Epot=  0.00025000000015636153  Etot=  0.0005000015625782497  q=  0.7070714260686285  p=  -0.005303256664678722  p0_dia=  1.0  p0_adi=  1.0
step=  2  Ekin=  0.0002500015622656477  Epot=  0.0002500000003517117  Etot=  0.0005000015626173594  q=  0.7070272329997399  p=  -0.008838613794993148  p0_dia=  1.0  p0_adi=  1.0
step=  3  Ekin=  0.00025000156203130715  Epot=  0.0002500000006251523  Etot=  0.0005000015626564594  q=  0.7069653642500252  p=  -0.012373749959962675  p0_dia=  1.0  p0_adi=  1.0
step=  4  Ekin=  0.00025000156171888855  Epot=  0.00025000000097665496  Etot=  0.0005000015626955435  q=  0.7068858213662036  p=  -0.01590857678118353  p0_dia=  1.0  p0_adi=  1.0
step=  5  Ekin=  0.0002500015613284251  Epot=  0.0002500000014061854  Etot=  0.0005000015627346106  q=  0.

step=  46  Ekin=  0.00025000147955722013  Epot=  0.00025000008471562645  Etot=  0.0005000015642728466  q=  0.6876714452579759  p=  -0.16292532921375202  p0_dia=  1.0  p0_adi=  1.0
step=  47  Ekin=  0.0002500014760190342  Epot=  0.00025000008828865124  Etot=  0.0005000015643076855  q=  0.6868396268258726  p=  -0.16636368644001276  p0_dia=  1.0  p0_adi=  1.0
step=  48  Ekin=  0.00025000147241137187  Epot=  0.0002500000919309719  Etot=  0.0005000015643423438  q=  0.6859906374030943  p=  -0.16979788457411277  p0_dia=  1.0  p0_adi=  1.0
step=  49  Ekin=  0.0002500014687345952  Epot=  0.0002500000956422249  Etot=  0.0005000015643768202  q=  0.6851244982143825  p=  -0.17322783776109935  p0_dia=  1.0  p0_adi=  1.0
step=  50  Ekin=  0.00025000146498907043  Epot=  0.00025000009942203945  Etot=  0.0005000015644111099  q=  0.6842412309132168  p=  -0.17665346025214185  p0_dia=  1.0  p0_adi=  1.0
step=  51  Ekin=  0.000250001461175173  Epot=  0.0002500001032700364  Etot=  0.0005000015644452095  q=  

step=  95  Ekin=  0.0002500012325142549  Epot=  0.00025000033319521546  Etot=  0.0005000015657094703  q=  0.6271999615204569  p=  -0.32495847973258396  p0_dia=  1.0  p0_adi=  1.0
step=  96  Ekin=  0.0002500012261143066  Epot=  0.00025000033961753717  Etot=  0.0005000015657318438  q=  0.6255594891228545  p=  -0.32809447954015813  p0_dia=  1.0  p0_adi=  1.0
step=  97  Ekin=  0.0002500012196698732  Epot=  0.00025000034608402223  Etot=  0.0005000015657538955  q=  0.6239033777380277  p=  -0.3312222769857453  p0_dia=  1.0  p0_adi=  1.0
step=  98  Ekin=  0.0002500012131815992  Epot=  0.00025000035259402495  Etot=  0.0005000015657756242  q=  0.622231668768758  p=  -0.3343417938744082  p0_dia=  1.0  p0_adi=  1.0
step=  99  Ekin=  0.0002500012066501333  Epot=  0.00025000035914689317  Etot=  0.0005000015657970264  q=  0.6205444040077698  p=  -0.33745295221822547  p0_dia=  1.0  p0_adi=  1.0


### Exercise 4

Write the scripts to visualize various quantities computed by the dynamics

### Exercise 5

Compute the population in a certain region of space and observe how it evolves during the dynamics

### Exercise 6

Compute the dynamics of the 2D wavepacked we set up in the above examples.

### Exercise 7

Explore the behavior of the dynamics (e.g. conservation of energy, etc.) as you vary the initial conditions (e.g. the parameters of the initial wavefunction), the integration parameters (e.g. dt), and the grid properties (grid spacing and the boundaries)